In [1]:
import pandas as pd
import numpy as np
from nltk import *
from sklearn.preprocessing import LabelEncoder

In [2]:
#Details of the features--
#https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [106]:
df = pd.read_csv('SpotifyFeatures.csv')
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [107]:
df = df[df['popularity']>55]
len(df)

53736

In [108]:
df = df[~df.duplicated(subset= ['track_name','artist_name' ])]
len(df)

24826

In [109]:
df.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [110]:
# df.describe()

In [111]:
df = df[[ 'track_name', 'genre', 'artist_name', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'valence']]
df['duration_ms'] = df['duration_ms'] //1000
df.rename(columns={"duration_ms": "duration_sec"}, inplace=True)
#df.head()

In [112]:
def numbers_normalised(df, col_list):
    for col in col_list:
        maxx = df[col].max()
        minn = df[col].min()
        df[col] = round((df[col] - minn) / (maxx - minn), 3) 

numbers_normalised(df, ['popularity', 'loudness', 'duration_sec', 'tempo'])

In [113]:
# df[df['loudness']==3.744000]

In [114]:
le = LabelEncoder()
df['mode'] = le.fit_transform(df['mode'])

In [115]:
df.head()

,track_name,genre,artist_name,popularity,acousticness,danceability,duration_sec,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence
135,Be Without You - Kendu Mix,R&B,Mary J. Blige,0.205,0.0830,0.724,0.156,0.689,0.000000,0.3040,0.832,1,0.1350,0.607,0.6930
136,Desperado,R&B,Rihanna,0.159,0.3230,0.685,0.112,0.610,0.000000,0.1020,0.848,1,0.0439,0.320,0.3230
137,Ice On My Baby (feat. Kevin Gates) - Remix,R&B,Yung Bleu,0.136,0.0675,0.762,0.122,0.520,0.000004,0.1140,0.848,1,0.0959,0.214,0.0862
138,Heaven Falls / Fall on Me,R&B,Surfaces,0.114,0.3600,0.563,0.151,0.366,0.002430,0.0955,0.809,1,0.1210,0.271,0.7680
139,Love Myself,R&B,Olivia O'Brien,0.273,0.5960,0.653,0.132,0.621,0.000000,0.0811,0.836,1,0.0409,0.351,0.4660


In [119]:
df['numerical'] =  df[['acousticness',
       'danceability', 'duration_sec', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
        'valence']].sum(axis=1) + df['popularity']*2
df['numerical'].head()

135    5.633000
136    4.684900
137    4.101604
138    4.734930
139    5.323000
Name: numerical, dtype: float64

In [120]:
# Processing text data
df['textual'] = df['artist_name'] + " " + df['genre'] + " " + df['track_name']
df['textual'].head()

135         Mary J. Blige R&B Be Without You - Kendu Mix
136                                Rihanna R&B Desperado
137    Yung Bleu R&B Ice On My Baby (feat. Kevin Gate...
138               Surfaces R&B Heaven Falls / Fall on Me
139                       Olivia O'Brien R&B Love Myself
Name: textual, dtype: object

In [122]:
final_df = df[['track_name', 'textual', 'numerical']]
#final_df.head()
final_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
final_df.head()

,track_name,textual,numerical
0,Be Without You - Kendu Mix,Mary J. Blige R&B Be Without You - Kendu Mix,5.633000
1,Desperado,Rihanna R&B Desperado,4.684900
2,Ice On My Baby (feat. Kevin Gates) - Remix,Yung Bleu R&B Ice On My Baby (feat. Kevin Gate...,4.101604
3,Heaven Falls / Fall on Me,Surfaces R&B Heaven Falls / Fall on Me,4.734930
4,Love Myself,Olivia O'Brien R&B Love Myself,5.323000


In [123]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
cv = CountVectorizer() #ngram_range=(1,1)
text_vectorised = cv.fit_transform(final_df['textual']).toarray()
#cv.vocabulary_

In [125]:
cos_sim = cosine_similarity(text_vectorised,text_vectorised)

MemoryError: 

In [105]:
song = 'Love Yourself'
song_idx = final_df[final_df['track_name'] == song].index
all_top_10 = [] #pd.Series(np.zeros((len(song_idx)*10)))
count = 0
x = pd.Series()

for idx in song_idx:
    
    #print(idx)
    text_10 = pd.Series(cos_sim[idx]).sort_values(ascending=False)[:10]
    text_idx = text_10.index
    text_recom = (text_10 + final_df['numerical'].loc[text_idx]).sort_values(ascending=False)
    x = pd.concat((x, text_recom), axis=0).sort_values(ascending=False)
    #all_top_10.append(text_recom)
final_idx = x[:10].index
final_df.iloc[x[:10].index]

,track_name,textual,numerical
2558,What Do You Mean?,Justin Bieber Dance,6.388998
17317,Love Yourself,Justin Bieber Pop,5.599000
6112,We Are,Justin Bieber Dance,4.845000
3907,I'll Show You,Justin Bieber Dance,4.630163
21468,Nothing Like Us - Bonus Track,Justin Bieber Pop,4.257651
17863,Purpose,Justin Bieber Pop,4.203700
3916,Never Say Never,Justin Bieber Dance,4.385800
21385,Mark My Words,Justin Bieber Pop,3.891302
21203,Life Is Worth Living,Justin Bieber Pop,3.783900
21661,Caving (feat. James Droll),Justin Caruso Pop,3.462704


In [104]:
# all_top_10 = pd.Series(np.zeros((len(song_idx)*10)))
# all_top_10.shape